In [ ]:
# QSAR-Guided Docking Pipeline for Lactam-Based HDAC2 Inhibitors

This Jupyter Notebook contains the automated workflow used to prepare ligands and perform molecular docking with **AutoDock Vina**.  
It was developed to reproduce the **HDAC2 docking** results reported in the manuscript:

**From QSAR to Dynamics: A Data-Driven Design of Drug-Like Lactam-Based HDAC Inhibitors for Diffuse Large B-Cell Lymphoma**

---

## Contents
1. **Ligand Preparation** – Batch conversion of `.pdb` ligands to `.pdbqt` format using MGLTools.
2. **Automated Docking** – High-throughput docking with AutoDock Vina, logging best binding affinities.
3. **Results Export** – CSV file summarizing binding affinities for all compounds.

---

## Requirements
- **MGLTools 1.5.6** (for ligand preparation)
- **AutoDock Vina** (for docking)
- **Python 3.x**
- Required Python libraries: `os`, `glob`, `subprocess`

---

## Input Files
- `data/28_compounds.sdf` – Original ligand structures  
- Prepared `.pdb` files for ligands (S1–S28)  
- `receptor.pdbqt` – Prepared receptor structure  
- `config.txt` – AutoDock Vina configuration file  

---

## Output Files
- `docking_results/` – Docked ligand files and logs
- `docking_summary.csv` – Best binding affinity for each ligand


In [ ]:
# Batch ligand preparation using MGLTools for AutoDock
# Converts S1–S28 .pdb files to .pdbqt format

import os
import subprocess

# Define paths
input_folder_path = r"C:\path\to\your\pdb_files"  # folder containing S1.pdb, S2.pdb, ...
output_folder_path = r"C:\path\to\output_pdbqt"   # folder to save .pdbqt files
mgltools_python_path = r"C:\Program Files (x86)\MGLTools-1.5.6\python.exe"
prepare_ligand_script = r"C:\Program Files (x86)\MGLTools-1.5.6\Lib\site-packages\AutoDockTools\Utilities24\prepare_ligand4.py"

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process all PDB files
for i in range(1, 29):  # S1 to S28
    pdb_file = os.path.join(input_folder, f"S{i}.pdb")
    pdbqt_file = os.path.join(output_folder, f"lig{i}.pdbqt")

    if os.path.exists(pdb_file):
        print(f"Processing {pdb_file} ...")

        command = [
            mgltools_python, prepare_ligand_script,
            "-l", pdb_file,
            "-o", pdbqt_file,
            "-A", "hydrogens"
        ]

        try:
            result = subprocess.run(command, shell=True, capture_output=True, text=True)
            print(result.stdout)  # Success messages
            print(result.stderr)  # Error messages if any
        except Exception as e:
            print(f"Error processing {pdb_file}: {e}")
    else:
        print(f"File {pdb_file} not found. Skipping.")

print("Batch conversion complete!")


In [ ]:
# Automated docking using AutoDock Vina
# Loops through all ligands and logs best affinity values
import subprocess
import os
import glob

# ==== USER-DEFINED PATHS ====
vina_path = r"C:\path\to\vina.exe"  # AutoDock Vina executable
receptor_path = r"C:\path\to\receptor.pdbqt"  # Prepared receptor file
ligand_folder_path = r"C:\path\to\ligands_pdbqt"  # Folder with .pdbqt ligands
output_folder_path = r"C:\path\to\docking_results"  # Where outputs will be saved
config_file_path = r"C:\path\to\config.txt"  # Vina config file
summary_file = os.path.join(output_folder, "docking_summary.csv")

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all ligand files in the folder
ligands = glob.glob(os.path.join(ligand_folder, "*.pdbqt"))

# Check if Vina and receptor exist
if not os.path.exists(vina_path):
    print("Error: AutoDock Vina executable not found.")
    exit(1)
if not os.path.exists(receptor):
    print("Error: Receptor file not found.")
    exit(1)
if not os.path.exists(config):
    print("Error: Configuration file not found.")
    exit(1)

# Prepare summary file
with open(summary_file, "w") as f:
    f.write("Ligand,Best_Affinity (kcal/mol)\n")

# Run docking for each ligand
for ligand in ligands:
    ligand_name = os.path.basename(ligand).replace(".pdbqt", "")
    output_file = os.path.join(output_folder, f"{ligand_name}_out.pdbqt")
    log_file = os.path.join(output_folder, f"{ligand_name}_log.txt")

    # Define the Vina command
    vina_command = [
        vina_path,
        "--receptor", receptor,
        "--ligand", ligand,
        "--out", output_file,
        "--log", log_file,
        "--config", config
    ]

    # Run AutoDock Vina
    try:
        print(f"Docking {ligand_name}...")
        subprocess.run(vina_command, check=True)
        print(f"Docking completed for {ligand_name}.")
    except subprocess.CalledProcessError:
        print(f"Error during docking for {ligand_name}. Skipping...")
        continue

    # Parse the log file to extract the best binding affinity
    best_affinity = None
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            for line in f:
                if line.strip().startswith("-----+"):
                    next_line = next(f, None)
                    if next_line:
                        best_affinity = float(next_line.split()[1])  # Extract affinity from the first result
                        break

    # Save the best result
    with open(summary_file, "a") as f:
        f.write(f"{ligand_name},{best_affinity}\n")

print("Batch docking completed! Check 'docking_summary.csv' for results.")
